# Склейка train и test с features

In [2]:
import numpy as np
import pandas as pd
import dask.dataframe as dd

In [3]:
data_train = dd.read_csv('data/data_train.csv')
data_train.head(5)

,Unnamed: 0,id,vas_id,buy_time,target
0,0,540968,8.0,1537131600,0.0
1,1,1454121,4.0,1531688400,0.0
2,2,2458816,1.0,1534107600,0.0
3,3,3535012,5.0,1535922000,0.0
4,4,1693214,1.0,1535922000,0.0


In [4]:
# data_train = data_train.drop('Unnamed: 0', axis=1)
# data_train.head(3)

In [5]:
data_test = dd.read_csv('data/data_test.csv')
data_test.head(3)

,Unnamed: 0,id,vas_id,buy_time
0,0,3130519,2.0,1548018000
1,1,2000860,4.0,1548018000
2,2,1099444,2.0,1546808400


In [6]:
features = dd.read_csv('data/features.csv', sep='\t')

In [7]:
features.head(3)

,Unnamed: 0,id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
0,0,2013026,1531688400,18.910029,46.980888,4.969214,-1.386798,3.791754,-14.01179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-301.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,1,2014722,1539550800,36.690029,152.400888,448.069214,563.833202,463.841754,568.99821,-16.08618,...,-891.373846,-544.770792,-20.996269,48.369552,80.252276,-13.832889,-0.694428,-1.175933,-0.45614,0.0
2,2,2015199,1545598800,-67.019971,157.050888,-63.180786,178.103202,-68.598246,156.99821,3.51382,...,-977.373846,-613.770792,-12.996269,-37.630448,10829.252276,-25.832889,-0.694428,-12.175933,-0.45614,0.0


In [8]:
# features.shape[0].compute()

# 4512528

In [9]:
# features = features.drop('Unnamed: 0', axis=1)

In [10]:
# data_train = data_train.set_index('Unnamed: 0', sorted=False)
# data_test = data_test.set_index('Unnamed: 0', sorted=False)
# features = features.set_index('Unnamed: 0', sorted=False)

In [11]:
ids = np.unique(data_train['id'].append(data_test['id']))
features = features[features['id'].isin(ids)]

In [12]:
data_train = data_train.compute()
data_test = data_test.compute()
features = features.compute()

In [13]:
data_train.drop('Unnamed: 0', axis=1, inplace=True)
data_test.drop('Unnamed: 0', axis=1, inplace=True)
features.drop('Unnamed: 0', axis=1, inplace=True)

In [14]:
data_train.sort_values(by='id', inplace=True)
data_test.sort_values(by='id', inplace=True)
features.sort_values(by='id', inplace=True)

In [15]:
features.head(3)

,id,buy_time,0,1,2,3,4,5,6,7,...,243,244,245,246,247,248,249,250,251,252
9741,2,1545598800,-96.799971,229.530888,-110.740786,305.723202,-116.158246,284.61821,-16.08618,-65.076097,...,2300.626154,1492.229208,-21.996269,-35.630448,368.252276,11.167111,7.305572,-12.175933,-0.45614,0.0
2301,4,1538341200,-96.799971,-323.429112,-110.740786,-376.036798,-116.158246,-397.14179,-16.08618,19.673903,...,4806.626154,4140.229208,-25.996269,-37.630448,-35.747724,-20.832889,-0.694428,-7.175933,-0.45614,0.0
10004,15,1531688400,260.810029,186.710888,249.409214,140.873202,243.991754,119.76821,341.52382,317.953903,...,-381.373846,-613.770792,-24.996269,-31.630448,-212.747724,108.167111,-0.694428,107.824067,13.54386,1.0


In [16]:
def merge_purchases_with_features(data, feats):
    # return pd.merge(data, feats.drop_duplicates('id'), how='left', on='id')
    return pd.merge_asof(data, feats, by='buy_time', on='id', direction='nearest')

In [17]:
df_data_train = merge_purchases_with_features(data_train, features)
df_data_train.head(5)

,id,vas_id,buy_time,target,0,1,2,3,4,5,...,243,244,245,246,247,248,249,250,251,252
0,2,2.0,1545598800,0.0,-96.799971,229.530888,-110.740786,305.723202,-116.158246,284.61821,...,2300.626154,1492.229208,-21.996269,-35.630448,368.252276,11.167111,7.305572,-12.175933,-0.45614,0.0
1,4,1.0,1533502800,0.0,-42.559971,99.460888,-43.780786,72.293202,-49.198246,51.18821,...,-927.373846,-570.770792,-24.996269,-36.630448,-212.747724,-14.832889,-0.694428,-1.175933,-0.45614,1.0
2,15,1.0,1534107600,0.0,-96.799971,-336.159112,-110.740786,-329.456798,-116.158246,-350.56179,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
3,16,2.0,1540760400,0.0,155.310029,-92.509112,158.309214,-95.126798,152.891754,-116.23179,...,-977.373846,-613.770792,-25.996269,-37.630448,-291.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
4,29,1.0,1533502800,0.0,-42.559971,99.460888,-43.780786,72.293202,-49.198246,51.18821,...,-927.373846,-570.770792,-24.996269,-36.630448,-212.747724,-14.832889,-0.694428,-1.175933,-0.45614,1.0


In [18]:
(df_data_train.shape[0], data_train.shape[0])

(831653, 831653)

In [19]:
X_test = merge_purchases_with_features(data_test, features)
X_test.head(3)

,id,vas_id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
0,55,2.0,1547413200,-65.299971,71.210888,-75.070786,35.273202,-80.488246,14.16821,-16.08618,...,-551.373846,-209.770792,-24.996269,-37.630448,-133.747724,-20.832889,-0.694428,-12.175933,-0.45614,1.0
1,64,4.0,1548018000,152.050029,257.220888,195.609214,267.113202,190.191754,246.00821,-16.08618,...,3185.626154,-613.770792,-21.996269,-37.630448,-139.747724,-24.832889,-0.694428,-12.175933,-0.45614,0.0
2,151,2.0,1547413200,-65.299971,71.210888,-75.070786,35.273202,-80.488246,14.16821,-16.08618,...,-551.373846,-209.770792,-24.996269,-37.630448,-133.747724,-20.832889,-0.694428,-12.175933,-0.45614,1.0


In [20]:
X = df_data_train.drop('target', axis=1).copy()
y = df_data_train['target'].copy()

In [21]:
X.to_csv('my_data/X.csv', index=False)
y.to_csv('my_data/y.csv', index=False)
X_test.to_csv('my_data/test.csv', index=False)

In [22]:
# X.head(3)

In [23]:
# y.head(5)

In [24]:
# https://stackoverflow.com/a/28882020/5468048

# def split_dataframe(df, chunk_size = 10000): 
#     chunks = list()
#     num_chunks = len(df) // chunk_size + 1
#     for i in range(num_chunks):
#         chunks.append(df[i*chunk_size:(i+1)*chunk_size])
#     return chunks

In [25]:
# X_chunks = split_dataframe(X, X.shape[0])
# len(X_chunks)

In [26]:
# from dask_ml.model_selection import train_test_split

In [27]:
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=1, shuffle=True)

In [28]:
# from dask_ml.linear_model import LogisticRegression

In [29]:
# X_train_a = X_train.to_dask_array()
# X_valid_a = X_valid.to_dask_array()
# y_train_a = y_train.to_dask_array()
# y_valid_a = y_valid.to_dask_array()

https://stackoverflow.com/q/56474033/5468048

In [30]:
# logr = LogisticRegression(solver_kwargs={"normalize":False})
# logr.fit(X_train_a, y_train_a)

In [31]:
# X_valid_a.compute_chunk_sizes()

In [32]:
# pred = logr.predict(X_valid_a)

In [33]:
# X_valid_a.shape

In [34]:
# exit()